In [44]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import allmusicUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from ioUtils import getFile, saveFile
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-26 23:02:58.070521


In [3]:
%load_ext autoreload
%autoreload
disc   = discogs('allmusic')
dutils = allmusicUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/artists-allmusic exists
/Volumes/Music/Discog/albums-allmusic exists
/Volumes/Music/Discog/artists-allmusic-db exists
/Volumes/Music/Discog/albums-allmusic-db exists
/Volumes/Music/Discog/artists-allmusic-db/metadata exists
/Volumes/Music/Discog/albums-allmusic-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Download Artist Data

In [3]:
artistIDToRef = disc.getArtistIDToRefData()

Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 16.2MB.
Loading /Volumes/Music/Discog/db/ArtistIDToRef.p


In [7]:
myMusicAlbumIDMap = getFile("/Users/tgadfort/Documents/code/music/musicDiscogsMap.p")

In [13]:
for artistName,artistVals in myMusicAlbumIDMap.items():
    artistID,discogsName = artistVals
    #arts.rmIDFiles(artistID)
    #artistID = '12596' # James Brown
    #ref      = artistIDToRef['12596']
    #arts.downloadArtistFromID(artistID, ref)

In [14]:
artistIDs = [v[0] for k,v in myMusicAlbumIDMap.items()]
arts.rmIDsFromDBs(artistIDs)

  Removed ArtistID 10390
  Removed ArtistID 365190
  Removed ArtistID 2347090
  Removed ArtistID 10990
  Removed ArtistID 1015890
  Removed ArtistID 257290
  Removed ArtistID 278390
  Removed ArtistID 238890
  Removed ArtistID 629790
  Removed ArtistID 76490
  Removed ArtistID 381390
  Removed ArtistID 960390
  Removed ArtistID 174890
  Removed ArtistID 1125690
  Removed ArtistID 818290
  Removed ArtistID 32990
  Removed ArtistID 237890
  Removed ArtistID 58690
  Removed ArtistID 117590
  Removed ArtistID 12590
  Removed ArtistID 2523290
  Removed ArtistID 238690
  Removed ArtistID 13690
  Removed ArtistID 3788090
  Removed ArtistID 291690
  Removed ArtistID 2136290
  Removed ArtistID 20890
  Removed ArtistID 48090
  Removed ArtistID 269090
  Removed ArtistID 2290
  Removed ArtistID 29890
  Removed ArtistID 787790
  Removed ArtistID 1351390
  Removed ArtistID 999990
  Removed ArtistID 2492090
  Removed ArtistID 58490
  Removed ArtistID 444890
  Removed ArtistID 253290
  Removed ArtistI

Saving 7053 artists to /Volumes/Music/Discog/artists-db/2-DB.p...   --> This file is 12.8MB.
Done.
  Removed ArtistID 15261
  Removed ArtistID 315661
  Removed ArtistID 254261
  Removed ArtistID 266461
  Removed ArtistID 307361
  Removed ArtistID 444961
  Removed ArtistID 353961
  Removed ArtistID 3419661
  Removed ArtistID 1048161
  Removed ArtistID 17961
  Removed ArtistID 242761
  Removed ArtistID 374761
  Removed ArtistID 23261
  Removed ArtistID 16761
  Removed ArtistID 107561
  Removed ArtistID 292261
  Removed ArtistID 234461
  Removed ArtistID 1666161
  Removed ArtistID 254761
  Removed ArtistID 38661
  Removed ArtistID 380261
  Removed ArtistID 251861
  Removed ArtistID 171561
  Removed ArtistID 405361
  Removed ArtistID 590661
  Removed ArtistID 485261
  Removed ArtistID 252661
  Removed ArtistID 279361
  Removed ArtistID 370761
  Removed ArtistID 521961
  Removed ArtistID 7161
  Removed ArtistID 252161
  Removed ArtistID 38561
  Removed ArtistID 8961
  Removed ArtistID 66986

Saving 7014 artists to /Volumes/Music/Discog/artists-db/88-DB.p...   --> This file is 12.0MB.
Done.
  Removed ArtistID 1798526
  Removed ArtistID 302126
  Removed ArtistID 93826
  Removed ArtistID 252326
  Removed ArtistID 261026
  Removed ArtistID 288326
  Removed ArtistID 205926
  Removed ArtistID 272426
  Removed ArtistID 292126
  Removed ArtistID 305926
  Removed ArtistID 313726
  Removed ArtistID 588226
  Removed ArtistID 27526
  Removed ArtistID 10226
  Removed ArtistID 391826
  Removed ArtistID 201126
  Removed ArtistID 46726
  Removed ArtistID 13726
  Removed ArtistID 3866926
  Removed ArtistID 254126
  Removed ArtistID 374226
  Removed ArtistID 17926
  Removed ArtistID 165426
  Removed ArtistID 40026
  Removed ArtistID 964526
  Removed ArtistID 17826
  Removed ArtistID 96826
  Removed ArtistID 3189626
  Removed ArtistID 1679926
  Removed ArtistID 1876126
  Removed ArtistID 261426
  Removed ArtistID 1472426
  Removed ArtistID 426526
  Removed ArtistID 276326
  Removed ArtistID 

Saving 7243 artists to /Volumes/Music/Discog/artists-db/81-DB.p...   --> This file is 12.8MB.
Done.
  Removed ArtistID 48424
  Removed ArtistID 17224
  Removed ArtistID 637924
  Removed ArtistID 260124
  Removed ArtistID 334724
  Removed ArtistID 253324
  Removed ArtistID 230624
  Removed ArtistID 361424
  Removed ArtistID 2561924
  Removed ArtistID 7624
  Removed ArtistID 1181524
  Removed ArtistID 355924
  Removed ArtistID 7170924
  Removed ArtistID 106624
  Removed ArtistID 75624
  Removed ArtistID 536424
  Removed ArtistID 8024
  Removed ArtistID 422724
  Removed ArtistID 66824
  Removed ArtistID 548424
  Removed ArtistID 2013724
  Removed ArtistID 248124
  Removed ArtistID 5229124
  Removed ArtistID 1024624
  Removed ArtistID 277224
  Removed ArtistID 329424
  Removed ArtistID 101424
  Removed ArtistID 4102924
  Removed ArtistID 1423824
  Removed ArtistID 629424
  Removed ArtistID 37224
  Removed ArtistID 1169124
  Removed ArtistID 514824
  Removed ArtistID 193124
  Removed Artist

Saving 7111 artists to /Volumes/Music/Discog/artists-db/8-DB.p...   --> This file is 12.2MB.
Done.
  Removed ArtistID 2770
  Removed ArtistID 391170
  Removed ArtistID 9070
  Removed ArtistID 263670
  Removed ArtistID 45070
  Removed ArtistID 351370
  Removed ArtistID 684370
  Removed ArtistID 57770
  Removed ArtistID 352770
  Removed ArtistID 2051470
  Removed ArtistID 323670
  Removed ArtistID 552270
  Removed ArtistID 3445870
  Removed ArtistID 2020670
  Removed ArtistID 536570
  Removed ArtistID 251870
  Removed ArtistID 180270
  Removed ArtistID 131970
  Removed ArtistID 171570
  Removed ArtistID 454970
  Removed ArtistID 166570
  Removed ArtistID 305870
  Removed ArtistID 293870
  Removed ArtistID 218270
  Removed ArtistID 11770
  Removed ArtistID 24170
  Removed ArtistID 1918370
  Removed ArtistID 288370
  Removed ArtistID 113470
  Removed ArtistID 259970
  Removed ArtistID 162670
  Removed ArtistID 59770
  Removed ArtistID 252170
  Removed ArtistID 414270
  Removed ArtistID 261

...

Saving 7157 artists to /Volumes/Music/Discog/artists-db/63-DB.p...   --> This file is 12.0MB.
Done.
  Removed ArtistID 3310737
  Removed ArtistID 264737
  Removed ArtistID 29737
  Removed ArtistID 2737
  Removed ArtistID 264437
  Removed ArtistID 276337
  Removed ArtistID 264637
  Removed ArtistID 37737
  Removed ArtistID 1470637
  Removed ArtistID 81037
  Removed ArtistID 382237
  Removed ArtistID 261637
  Removed ArtistID 291337
  Removed ArtistID 406537
  Removed ArtistID 18837
  Removed ArtistID 329937
  Removed ArtistID 45537
  Removed ArtistID 239937
  Removed ArtistID 90037
  Removed ArtistID 1017637
  Removed ArtistID 76037
  Removed ArtistID 448037
  Removed ArtistID 881137
  Removed ArtistID 967637
  Removed ArtistID 722937
  Removed ArtistID 4276637
  Removed ArtistID 2734037
  Removed ArtistID 268537
  Removed ArtistID 1377737
  Removed ArtistID 583437
  Removed ArtistID 1004737
  Removed ArtistID 39837
  Removed ArtistID 266437
  Removed ArtistID 825537
  Removed ArtistID 

Saving 7216 artists to /Volumes/Music/Discog/artists-db/92-DB.p...   --> This file is 12.3MB.
Done.
  Removed ArtistID 17479
  Removed ArtistID 266079
  Removed ArtistID 1412579
  Removed ArtistID 251679
  Removed ArtistID 1345479
  Removed ArtistID 1653779
  Removed ArtistID 99679
  Removed ArtistID 160079
  Removed ArtistID 283779
  Removed ArtistID 210879
  Removed ArtistID 82079
  Removed ArtistID 136179
  Removed ArtistID 3062379
  Removed ArtistID 278379
  Removed ArtistID 577579
  Removed ArtistID 47579
  Removed ArtistID 11879
  Removed ArtistID 347579
  Removed ArtistID 533479
  Removed ArtistID 38379
  Removed ArtistID 3671079
  Removed ArtistID 200979
  Removed ArtistID 2254179
  Removed ArtistID 252479
  Removed ArtistID 58479
  Removed ArtistID 1630979
  Removed ArtistID 3455979
  Removed ArtistID 77479
  Removed ArtistID 347479
  Removed ArtistID 3879
  Removed ArtistID 127679
  Removed ArtistID 1252279
  Removed ArtistID 254479
  Removed ArtistID 321579
  Removed ArtistI

Saving 7079 artists to /Volumes/Music/Discog/artists-db/55-DB.p...   --> This file is 12.4MB.
Done.
  Removed ArtistID 833315
  Removed ArtistID 199115
  Removed ArtistID 904215
  Removed ArtistID 2662615
  Removed ArtistID 417015
  Removed ArtistID 304715
  Removed ArtistID 45115
  Removed ArtistID 31615
  Removed ArtistID 4115
  Removed ArtistID 585715
  Removed ArtistID 161715
  Removed ArtistID 293315
  Removed ArtistID 293115
  Removed ArtistID 209415
  Removed ArtistID 537415
  Removed ArtistID 1639715
  Removed ArtistID 3317315
  Removed ArtistID 81815
  Removed ArtistID 39015
  Removed ArtistID 4015
  Removed ArtistID 996915
  Removed ArtistID 173115
  Removed ArtistID 485815
  Removed ArtistID 777915
  Removed ArtistID 29815
  Removed ArtistID 255515
  Removed ArtistID 495715
  Removed ArtistID 242415
  Removed ArtistID 269915
  Removed ArtistID 826415
  Removed ArtistID 907315
  Removed ArtistID 267715
  Removed ArtistID 734815
  Removed ArtistID 485215
  Removed ArtistID 194

Saving 7076 artists to /Volumes/Music/Discog/artists-db/49-DB.p...   --> This file is 11.8MB.
Done.
  Removed ArtistID 612401
  Removed ArtistID 125101
  Removed ArtistID 11101
  Removed ArtistID 254201
  Removed ArtistID 355001
  Removed ArtistID 1523501
  Removed ArtistID 521401
  Removed ArtistID 93801
  Removed ArtistID 156101
  Removed ArtistID 3101
  Removed ArtistID 377001
  Removed ArtistID 492401
  Removed ArtistID 215301
  Removed ArtistID 231601
  Removed ArtistID 378401
  Removed ArtistID 307601
  Removed ArtistID 112801
  Removed ArtistID 830701
  Removed ArtistID 11801
  Removed ArtistID 35301
  Removed ArtistID 21001
  Removed ArtistID 4081301
  Removed ArtistID 255801
  Removed ArtistID 56201
  Removed ArtistID 5201
  Removed ArtistID 2620801
  Removed ArtistID 674901
  Removed ArtistID 335901
  Removed ArtistID 85101
  Removed ArtistID 1157401
  Removed ArtistID 112901
  Removed ArtistID 116901
  Removed ArtistID 647601
  Removed ArtistID 4739001
  Removed ArtistID 274

Saving 7172 artists to /Volumes/Music/Discog/artists-db/78-DB.p...   --> This file is 12.4MB.
Done.


In [50]:
arts.parseArtistModValFiles(96, force=False)
arts.parseArtistModValExtraFiles(96)

Parsing Artist Files For ModVal 96
  ===> Found 7077 files. There are 1 new files to parse.
Saving 1 new artist IDs to /Volumes/Music/Discog/artists-db/96-DB.p
  --> This file is 12.2MB.
Saving 7077 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/96-Metadata.p
  --> This file is 413.5kB.
Saving 7077 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/96-MediaMetadata.p
  --> This file is 3.9MB.
Parsing Artist Extra Files For ModVal 96
  Found 8 extra files for ModVal 96
  Loaded 7077 artist IDs.
File: 0  	Artist: 12596 --> 500 to 1000
File: 1  	Artist: 12596 --> 1000 to 1030
File: 2  	Artist: 1864096 --> 837 to 837
File: 3  	Artist: 293896 --> 548 to 548
File: 4  	Artist: 359096 --> 1005 to 1005
File: 5  	Artist: 359096 --> 1005 to 1005
File: 6  	Artist: 604396 --> 585 to 585
File: 7  	Artist: 65796 --> 715 to 715
Saving 1720 new artist media to /Volumes/Music/Discog/artists-db/96-DB.p
  --> This file is 12.3MB.
Saving 7077 new artist IDs name d

1720

In [12]:
files = [savename]
from glob import glob
from os.path import join
from fileUtils import getDirname
files += glob(join(getDirname(savename), "extra", "{0}-*.p".format(artistID)))
files

['/Volumes/Music/Discog/artists/96/12596.p',
 '/Volumes/Music/Discog/artists/96/extra/12596-2.p',
 '/Volumes/Music/Discog/artists/96/extra/12596-3.p']

Removing /Volumes/Music/Discog/artists/96/12596.p
Removing /Volumes/Music/Discog/artists/96/extra/12596-2.p
Removing /Volumes/Music/Discog/artists/96/extra/12596-3.p


[None, None, None]

Now Downloading in Artists(): https://www.discogs.com/artist/12596-James-Brown?sort=year%2Casc&limit=500
                   Saving as: /Volumes/Music/Discog/artists/96/12596.p
Saving /Volumes/Music/Discog/artists/96/12596.p (force=False)
  --> This file is 158.8kB.
Done. Sleeping for 2 seconds


True

In [39]:
arts.rmIDFromDB(artistID)

Trying to remove data from ArtistID 12596
Loading /Volumes/Music/Discog/artists-db/96-DB.p
Not there...
Found 0 files to delete.
No reason to save /Volumes/Music/Discog/artists-db/96-DB.p


In [19]:
del db[artistID]

In [20]:
disc.saveArtistsDBModValData(96, db)

Saving 7076 artists to /Volumes/Music/Discog/artists-db/96-DB.p...   --> This file is 12.2MB.
Done.


In [22]:
arts.parseArtistModValFiles(96, force=False)

Parsing Artist Files For ModVal 96
  ===> Found 7077 files. There are 1 new files to parse.
Saving 1 new artist IDs to /Volumes/Music/Discog/artists-db/96-DB.p
  --> This file is 12.2MB.
Saving 7077 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/96-Metadata.p
  --> This file is 413.5kB.
Saving 7077 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/96-MediaMetadata.p
  --> This file is 3.9MB.


1

In [23]:
arts.assertDBModValExtraData(96)

Now Downloading in Artists(): https://www.discogs.com/artist/12596-James-Brown?sort=year%2Casc&limit=500&page=2
                   Saving as: /Volumes/Music/Discog/artists/96/extra/12596-2.p
Saving /Volumes/Music/Discog/artists/96/extra/12596-2.p (force=True)
  --> This file is 152.0kB.
Done. Sleeping for 2 seconds
Now Downloading in Artists(): https://www.discogs.com/artist/12596-James-Brown?sort=year%2Casc&limit=500&page=3
                   Saving as: /Volumes/Music/Discog/artists/96/extra/12596-3.p
Saving /Volumes/Music/Discog/artists/96/extra/12596-3.p (force=True)
  --> This file is 49.1kB.
Done. Sleeping for 2 seconds


In [26]:
#arts.parseArtistModValFiles(96, force=False)

Parsing Artist Extra Files For ModVal 96
  Found 8 extra files for ModVal 96
  Loaded 7077 artist IDs.
1000
1030
837
548
1005
1005
585
715
Saving 1720 new artist media to /Volumes/Music/Discog/artists-db/96-DB.p
  --> This file is 12.3MB.


1720

# Missing Artists From Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
from discogsBase import discogs

disc  = discogs()
albs  = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)

from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            print('\n{0}/{1}'.format(ia,len(refstoget)))
            retval = arts.downloadArtistURL(url, savename, debug=True)
            sleep(2)
        except:
            continue
    else:
        print("{0} known".format(savename))
            
    #print(item)
#len(togetmap)

# Artists From Search

In [4]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
arts.searchDiscogForArtist("Dababy")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


===================== Searching For Dababy =====================
Downloading: https://www.discogs.com/search/?q=Dababy&limit=250&type=artist
Found 2 artists
2 / 2 	: 7 	 https://www.discogs.com/artist/6619834-Dababy?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/34/6619834.p (force=False)
  --> This file is 18.5kB.
Done. Sleeping for 2 seconds


In [ ]:
from ioUtils import getFile

In [ ]:
toGet = getFile(ifile="/Users/tgadfort/Documents/code/charts/chartCounter.p", debug=True)

In [ ]:
toGet = getFile("~/code/charts/toget.p")

In [14]:
for i,val in enumerate(toGet):
    if i <= 206:
        continue
    print('='*30,val,'(',i,'/',len(toGet),')','='*30)
    arts.searchDiscogForArtist(val)
    print("\n\n\n")
    sleep(2)

============================== Ketnetband ( 167 / 473 ) ==============================


===================== Searching For Ketnetband =====================
Downloading: https://www.discogs.com/search/?q=Ketnetband&limit=250&type=artist
Found 2 artists
2 / 2 	: 6 	 https://www.discogs.com/artist/715720-Ketnet-Band?sort=year%2Casc&limit=500




============================== Sodapop ( 168 / 473 ) ==============================


===================== Searching For Sodapop =====================
Downloading: https://www.discogs.com/search/?q=Sodapop&limit=250&type=artist
Found 19 artists
2 / 19 	: 6 	 https://www.discogs.com/artist/421951-Sodapop-2?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/51/421951.p (force=False)
  --> This file is 16.2kB.
Done. Sleeping for 2 seconds
3 / 19 	: 7 	 https://www.discogs.com/artist/5182804-Sodapop-10?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/4/5182804.p (force=False)
  --> This file is 15.1kB.
Done. Sleeping for 2

  --> This file is 15.3kB.
Done. Sleeping for 2 seconds
14 / 70 	: 7 	 https://www.discogs.com/artist/6209599-Ron-Williams-And-The-Customs?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/99/6209599.p (force=False)
  --> This file is 14.9kB.
Done. Sleeping for 2 seconds
15 / 70 	: 7 	 https://www.discogs.com/artist/6074523-TC-Customs?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/23/6074523.p (force=False)
  --> This file is 14.9kB.
Done. Sleeping for 2 seconds
16 / 70 	: 7 	 https://www.discogs.com/artist/5995987-Customs-Officers?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/87/5995987.p (force=False)
  --> This file is 14.9kB.
Done. Sleeping for 2 seconds
17 / 70 	: 7 	 https://www.discogs.com/artist/4422387-Customs-Band?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/87/4422387.p (force=False)
  --> This file is 15.0kB.
Done. Sleeping for 2 seconds
18 / 70 	: 7 	 https://www.discogs.com/artist/3085709-Customstuff?sort=yea

58 / 70 	: 7 	 https://www.discogs.com/artist/7579602-Samuel-Henry-2?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/2/7579602.p (force=False)
  --> This file is 15.7kB.
Done. Sleeping for 2 seconds
59 / 70 	: 7 	 https://www.discogs.com/artist/3264350-Juan-Pulido?sort=year%2Casc&limit=500
60 / 70 	: 7 	 https://www.discogs.com/artist/3697554-Voltumna?sort=year%2Casc&limit=500
61 / 70 	: 7 	 https://www.discogs.com/artist/3759988-Berzsenyi-D%C3%A1niel?sort=year%2Casc&limit=500
62 / 70 	: 7 	 https://www.discogs.com/artist/3876876-Divanhana?sort=year%2Casc&limit=500
63 / 70 	: 7 	 https://www.discogs.com/artist/3635038-%D0%9E%D0%BA%D1%82%D0%B0%D0%B9?sort=year%2Casc&limit=500
64 / 70 	: 7 	 https://www.discogs.com/artist/1136121-Folklorni-Ansambl-Milica-Kri%C5%BEan?sort=year%2Casc&limit=500
65 / 70 	: 7 	 https://www.discogs.com/artist/6722308-Caramelles-Del-Roser?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/8/6722308.p (force=False)
  --> This file is 18

Saving /Volumes/Music/Discog/artists/33/5438833.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
35 / 36 	: 7 	 https://www.discogs.com/artist/5136636-Clean-12?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/36/5136636.p (force=False)
  --> This file is 16.4kB.
Done. Sleeping for 2 seconds
36 / 36 	: 7 	 https://www.discogs.com/artist/6327557-7Up-Ensemble?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/57/6327557.p (force=False)
  --> This file is 15.5kB.
Done. Sleeping for 2 seconds




============================== Matthijs Vanstaen ( 173 / 473 ) ==============================


===================== Searching For Matthijs Vanstaen =====================
Downloading: https://www.discogs.com/search/?q=Matthijs%20Vanstaen&limit=250&type=artist
Found 3 artists
2 / 3 	: 7 	 https://www.discogs.com/artist/1418537-Matthijs-Vanstaen?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/37/1418537.p (force=False)
  --> This file i

6 / 251 	: 6 	 https://www.discogs.com/artist/302542-Access-7?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/42/302542.p (force=False)
  --> This file is 15.9kB.
Done. Sleeping for 2 seconds
7 / 251 	: 7 	 https://www.discogs.com/artist/4210303-All-Access-Entertainment?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/3/4210303.p (force=False)
  --> This file is 22.3kB.
Done. Sleeping for 2 seconds
8 / 251 	: 5 	 https://www.discogs.com/artist/37653-The-Easy-Access-Orchestra?sort=year%2Casc&limit=500
9 / 251 	: 7 	 https://www.discogs.com/artist/3770538-All-Access-Agency?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/38/3770538.p (force=False)
  --> This file is 19.2kB.
Done. Sleeping for 2 seconds
10 / 251 	: 6 	 https://www.discogs.com/artist/199561-Access_Denied?sort=year%2Casc&limit=500
11 / 251 	: 6 	 https://www.discogs.com/artist/439189-Random-Access-2?sort=year%2Casc&limit=500
12 / 251 	: 7 	 https://www.discogs.com/artist/1098421-Ac

...

190 / 251 	: 6 	 https://www.discogs.com/artist/819678-Ruby-Jean-And-The-Thoughtful-Bees?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/78/819678.p (force=False)
  --> This file is 16.7kB.
Done. Sleeping for 2 seconds
191 / 251 	: 7 	 https://www.discogs.com/artist/4446523-Maurice-Beeson?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/23/4446523.p (force=False)
  --> This file is 15.3kB.
Done. Sleeping for 2 seconds
192 / 251 	: 7 	 https://www.discogs.com/artist/1883884-See-Bees?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/84/1883884.p (force=False)
  --> This file is 15.0kB.
Done. Sleeping for 2 seconds
193 / 251 	: 7 	 https://www.discogs.com/artist/3653172-Bees-In-Trees?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/72/3653172.p (force=False)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
194 / 251 	: 7 	 https://www.discogs.com/artist/2524856-Bumble-Bees-2?sort=year%2Casc&limit=500
Saving /Volumes/Music/Disc

230 / 251 	: 7 	 https://www.discogs.com/artist/5746715-Paul-Beesley-2?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/15/5746715.p (force=False)
  --> This file is 14.9kB.
Done. Sleeping for 2 seconds
231 / 251 	: 7 	 https://www.discogs.com/artist/1141186-Het-Beest?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/86/1141186.p (force=False)
  --> This file is 15.5kB.
Done. Sleeping for 2 seconds
232 / 251 	: 7 	 https://www.discogs.com/artist/6029072-Steve-Beeson-2?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/72/6029072.p (force=False)
  --> This file is 16.2kB.
Done. Sleeping for 2 seconds
233 / 251 	: 7 	 https://www.discogs.com/artist/3938983-Johnny-Huskey-And-The-King-Bees?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/83/3938983.p (force=False)
  --> This file is 16.2kB.
Done. Sleeping for 2 seconds
234 / 251 	: 7 	 https://www.discogs.com/artist/4012247-Bee-Beeson?sort=year%2Casc&limit=500
Saving /Volumes/Music/Disco

7 / 27 	: 7 	 https://www.discogs.com/artist/2668811-Peter-Zimmers?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/11/2668811.p (force=False)
  --> This file is 15.0kB.
Done. Sleeping for 2 seconds
8 / 27 	: 7 	 https://www.discogs.com/artist/2093329-Steve-Zimmers?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/29/2093329.p (force=False)
  --> This file is 14.9kB.
Done. Sleeping for 2 seconds
9 / 27 	: 7 	 https://www.discogs.com/artist/3742178-Bill-Zimmers?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/78/3742178.p (force=False)
  --> This file is 15.2kB.
Done. Sleeping for 2 seconds
10 / 27 	: 7 	 https://www.discogs.com/artist/1381245-The-Zimmers?sort=year%2Casc&limit=500
11 / 27 	: 7 	 https://www.discogs.com/artist/4094318-Udo-Zimmerschitt?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/18/4094318.p (force=False)
  --> This file is 14.9kB.
Done. Sleeping for 2 seconds
12 / 27 	: 7 	 https://www.discogs.com/artist/6361552

Downloading: https://www.discogs.com/search/?q=Precision%20Tunes&limit=250&type=artist
Found 8 artists
2 / 8 	: 7 	 https://www.discogs.com/artist/3610271-Artifact-6?sort=year%2Casc&limit=500
3 / 8 	: 7 	 https://www.discogs.com/artist/1622883-E-Clip?sort=year%2Casc&limit=500
4 / 8 	: 7 	 https://www.discogs.com/artist/1623609-Sgt-6-Assault?sort=year%2Casc&limit=500
5 / 8 	: 7 	 https://www.discogs.com/artist/1849607-Direct-4?sort=year%2Casc&limit=500
Saving /Volumes/Music/Discog/artists/7/1849607.p (force=False)
  --> This file is 20.8kB.
Done. Sleeping for 2 seconds
6 / 8 	: 7 	 https://www.discogs.com/artist/4712309-TRADArrr?sort=year%2Casc&limit=500
7 / 8 	: 7 	 https://www.discogs.com/artist/5276773-Poit%C3%ADn?sort=year%2Casc&limit=500
8 / 8 	: 6 	 https://www.discogs.com/artist/405253-Rain-Man?sort=year%2Casc&limit=500




============================== Isabell Schmidt ( 206 / 473 ) ==============================


===================== Searching For Isabell Schmidt ============

HTTPError: HTTP Error 404: NOT FOUND

In [ ]:
from time import sleep
vals  = ["The Souvenirs", "The Stand", f"The Suicide Kings", "The Supremes", "The Temptations", "The Tomcats", "The Tone", "The Tourists", "The Unseen", "The Ventures", "The Vikings", "The Violents", "The Wailers", "The Want", "The Washingtonians", "The Witches", "UB40", "Underdog", "Unearth", "Unk", "Unscarred", "Up Front", "Up In Arms", "Urban Legends"]
vals += ["The Pits", "The Pressure", "The Pretenders", "The Prodigy", "The Product", "The Receptionists", "The Retardos", "The Revelers", "The Romancers", "The Roots", "The Rosehips", "The Ruiners", "The Runaways", "The Scholars", "The Seeds", "The Shams", "The Sights", "The Sillies", "The Skunks", "The Snowmen"]
vals += ["The Invisible Men", "The Israelites", "The Ivory Coast", "The Killers", "The Manatees", "The Marquees", "The Masters", "The Matrix", "The Mixtures", "The Moves", "The Noisettes", "The Nomads", "The Orange Peels", "The Outcasts"]
vals += ["The Game", "The Generics", "The Gits", "The Go-Betweens", "The Goblins", "The Godz", "The Great Society", "The Haters", "The Hilltops", "The Hookers", "The Immortals", "The Impacts", "The Inspector", "The Invaders"]
vals += ["The Diamonds", "The Diplomats", "The Dramatics", "The Drifters", "The Dust Brothers", "The Eagles", "The Exceptions", "The Fiends", "The Fixtures", "The Frantics", "The Freeze", "The Frogs", "The Fumes"]
vals += ["The Cardigans", "The Casualties", "The Challengers", "The Chameleons", "The Cheaters", "The Chicken Hawks", "The Crash", "The Crusaders", "The Cure", "The Darts", "The Deuce Coupes"]
vals += ["The Beat", "The Beatles", "The Blood Brothers", "The Blow Up", "The Bollweevils", "The Brentwoods", "The Broadways", "The Brood", "The Business", "The Busy Signals"]
vals += ["You", "Young Money", "Young Pioneers", "Zero Zero"]
for val in vals:
    print('='*30,val,'='*30)
    try:
        arts.searchDiscogForArtist(val)
    except:
        continue
    print("\n\n\n")
    sleep(10)

# New Artists From Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums

disc = discogs()
albs = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

In [ ]:
from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    print('\n{0}/{1}'.format(ia,len(refstoget)))
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            arts.downloadArtistURL(url, savename, debug=True)
        except:
            continue
        sleep(2)
    #print(item)
#len(togetmap)

# Albums From Known People

In [ ]:
data = getFile("/Volumes/Music/MusicDB/toget.p")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

_ = clock("Last Run")

In [ ]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
toget={}
known={}

for ia,artistName in enumerate(data):
    #artistID = artistName
    artistID = None
    if artistID is None:
        retval = getMusicData('Name', artistName)
        if retval is not None:
            artistID = retval.index.values[0]
    modValue  = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
    if toget.get(modValue) is None:
        toget[modValue] = {}
    toget[modValue][artistID] = True
        
    if ia % 100 == 0:
        print(ia,'/',len(data))

In [ ]:
for modVal in toget.keys():
    artistIDs = toget[modVal]
    print(modVal,'\t',len(artistIDs))
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID in artistIDs:
        print(' ',artistID)
        artistData = dbdata[artistID]
        albs.downloadAlbumFromArtistData(artistID=artistID, artistData=artistData, mediaTypes=None)

# Albums From All Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=1, mediaTypes=None, rand=True)

# Artists From Collections

In [ ]:
%load_ext autoreload
%autoreload

from collection import Counter
refCounts = Counter(disc.getCollectionRefCountsData())

In [ ]:
artistIDDF = disc.getMasterArtistDiscogsDB()
print(artistIDDF["Ref"].shape)
artistRefs = {v: k for k, v in artistIDDF["Ref"].to_dict().items()}
artistIDs  = {k: v for k, v in artistIDDF["Ref"].to_dict().items()}
print(len(artistRefs))

In [ ]:
try:
    errFile = setFile(disc.getDiagnosticDir(), "CollectionErrors.json")
    errs = getFile(errFile)
except:
    raise ValueError("Problem loading collections file")
    
print("Found {0} errors".format(len(errs)))
nNext = len(errs)

n = len(refCounts)
nFalse = 0
nNone  = 0
nTrue  = 0

for i,(ref,cnts) in enumerate(refCounts.most_common()):
    if i < 286093:
        continue
    if i % 2500 == 0:
        print("Processed {0}/{1}  (Errs={2})".format(i,n, len(errs)))
        
    if errs.get(ref) is not None:
        continue
        
    artistID = dutils.getArtistID(ref)
    if artistID is None:
        continue
    if artistIDs.get(artistID) is None:
        url      = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        if isFile(savename):
            continue
            
        try:
            retval = arts.downloadArtistURL(url=url, savename=savename, parse=True, debug=True)
            sleep(1)
        except:
            retval = None
            
        if retval is None:
            nNone += 1
            errs[ref] = "None"
            print("\t--->Adding None for {0}".format(artistID))
        if retval is False:
            nFalse += 1
            errs[ref] = "False"
            print("\t--->Adding False for {0}".format(artistID))

            
        sleep(3)

    if i % 1000 == 0:
        print("There have been {0} errors so far...".format(nNone))
        print("There have been {0} no saves so far...".format(nFalse))
        print("There have been {0} full saves so far...".format(nTrue))
        
        errFile = setFile(disc.getDiagnosticDir(), "CollectionErrors.json")
        saveFile(ifile=errFile, idata=errs, debug=True)


            
    if len(errs) == nNext + 100:
        errFile = setFile(disc.getDiagnosticDir(), "CollectionErrors.json")
        saveFile(ifile=errFile, idata=errs, debug=True)
        print("Breaking after 250 errors")
        print("Last I = {0}".format(i))
        break

In [ ]:
len(errs)

In [ ]:
disc.getDiagnosticDir()

In [ ]:
## Search for

In [1]:
artistName = "Ellie Goulding"

In [34]:
#url = https://www.allmusic.com/search/artists/Ellie+Goulding
import urllib
from urllib.parse import quote
from webUtils import getHTML
from json import loads

In [7]:
url     = urllib.parse.urljoin(baseURL, quote(artistName))
#url     = urllib.parse.urljoin(url, "?sort=year%2Casc&limit=500") ## Make sure we get 500 entries)

'https://www.allmusic.com/search/artists/Ellie%20Goulding'

In [42]:
def getURLData(url, debug=True):    
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    if debug:
        print("Now Downloading: {0}".format(url))

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need
    
    return data, response

In [37]:
myMusicMap = getFile("/Users/tgadfort/Documents/code/music/musicDiscogsMap.p")

In [40]:
allMusicIDMap = {}

In [ ]:
baseURL = "https://www.allmusic.com/search/artists/"
for i,(artistName,artistVals) in enumerate(myMusicMap.items()):
    print(i,'\t',artistName)
    url     = urllib.parse.urljoin(baseURL, quote(artistName))

    data, response = getURLData(url)
    if response.code == 200:
        bsdata = getHTML(data)
    else:
        continue
        
    infos = bsdata.findAll("div", {"class": "info"})
    for info in infos:
        for name in info.findAll("div", {"class": "name"}):
            ref = name.find("a")
            if ref is None:
                continue
            href     = ref.attrs['href']
            artistID = loads(ref.attrs['data-tooltip'])['id']
            name     = ref.text
            print("\t{0: <70}{1: <20}".format(name,artistID))
            if allMusicIDMap.get(artistID) is None:
                allMusicIDMap[artistID] = [name, href]
                
    print("  -->",len(allMusicIDMap))
    sleep(2)
    if i % 50 == 0:
        savename = "collections/allMusicIDMap.p"
        print("Saving {0} AllMusic IDs to {1}".format(len(allMusicIDMap), savename))
        saveFile(idata=allMusicIDMap, ifile=savename, debug=True)

0 	 A Flock Of Seagulls
Now Downloading: https://www.allmusic.com/search/artists/A%20Flock%20Of%20Seagulls
	A Flock of Seagulls                                                   MN0000567862        
	The Angry Kids Vs. a Flock of Seagulls                                MN0002604767        
	Bp Vs. Effcee Vs. A Flock Of Seagulls                                 MN0000736869        
	Flock of Dimes                                                        MN0002921785        
	Flock of Doug                                                         MN0000698937        
	A Flock of Fantasy                                                    MN0003158085        
	The Professional Seagulls Dance Band of Port Harcourt                 MN0002420304        
  --> 7
Saving 7 AllMusic IDs to collections/allMusicIDMap.p
Saving data to collections/allMusicIDMap.p
  --> This file is 441B.
Saved data to collections/allMusicIDMap.p
  --> This file is 441B.
1 	 A Global Threat
Now Downloading: https://www.allm

6 	 A-Set
Now Downloading: https://www.allmusic.com/search/artists/A-Set
	A-Set                                                                 MN0000572590        
	Aset 1                                                                MN0002777203        
	The Aset Players                                                      MN0003746210        
	Aset SoSavvy                                                          MN0003834789        
	Badi Assad                                                            MN0000381739        
	Lords of Acid                                                         MN0000279718        
	Scratch Acid                                                          MN0000307332        
	Acid Mothers Temple                                                   MN0000585520        
	Uncle Acid & the Deadbeats                                            MN0002822383        
	Acid Bath                                                             MN0000592864        
	Acid K

14 	 Accept
Now Downloading: https://www.allmusic.com/search/artists/Accept
	Accept                                                                MN0000925837        
	Accept, Accept                                                        MN0000591393        
	Aspid                                                                 MN0001920610        
	Alessandro Aspide                                                     MN0002095420        
	Spiros Aspiotis                                                       MN0002975373        
	Konstantinos Aspiotis                                                 MN0003211926        
	Manuel Aspidi                                                         MN0003243777        
	Dionisis Aspiotis                                                     MN0003709488        
	Gust M. Aspiotes                                                      MN0002736919        
	Ensemble Nuovo Aspetto                                                MN0002994490        
  --

19 	 Adam Ant
Now Downloading: https://www.allmusic.com/search/artists/Adam%20Ant
	Adam Ant                                                              MN0000496487        
	Adam and the Ants                                                     MN0000590787        
	Adam Ant Anton                                                        MN0001461115        
	Brian Adam Ant                                                        MN0002169441        
	Satan & Adam                                                          MN0000303749        
	Anda Adam                                                             MN0002006431        
	Adam Larson & Co.                                                     MN0003827013        
	Adam & I                                                              MN0003283430        
	Adam & Eve                                                            MN0000591126        
	Adam & Stephanie                                                      MN0000631123       

24 	 Adryan Russ
Now Downloading: https://www.allmusic.com/search/artists/Adryan%20Russ
	Adryan Russ                                                           MN0000500904        
	Russ Taff                                                             MN0000212810        
	Russ Morgan                                                           MN0000176281        
	Russ                                                                  MN0003531552        
	Russ Freeman                                                          MN0000808588        
	Russ Barenberg                                                        MN0000805216        
	Russ Freeman                                                          MN0002444027        
	Russ Ballard                                                          MN0000206990        
	Russ Hartenberger                                                     MN0000210855        
	Russ Kunkel                                                           MN0000176293 

...

540 	 Chitlin' Fooks
Now Downloading: https://www.allmusic.com/search/artists/Chitlin%27%20Fooks
	Chitlin' Fooks                                                        MN0000773880        
  --> 7306
541 	 Chokebore
Now Downloading: https://www.allmusic.com/search/artists/Chokebore
	Chokebore                                                             MN0000087280        
  --> 7307
542 	 Choking Victim
Now Downloading: https://www.allmusic.com/search/artists/Choking%20Victim
	Choking Victim                                                        MN0000102964        
	Victim                                                                MN0000204687        
	Choking Ahogo                                                         MN0000842375        
	Session Victim                                                        MN0001093342        
	Victim Nation                                                         MN0001974650        
	Victim Mentality                                          

547 	 Choreboy
Now Downloading: https://www.allmusic.com/search/artists/Choreboy
	Choreboy                                                              MN0000087376        
	Chore-Boy                                                             MN0003853193        
	Choirboys                                                             MN0000069702        
	Cherubs                                                               MN0000111739        
	John Sherba                                                           MN0000683599        
	Cherub                                                                MN0002939023        
	The Shrubs                                                            MN0001601822        
	Shirabe Aoki                                                          MN0001653091        
	Charles Sherba                                                        MN0001658606        
	Consuelo Sherba                                                       MN0001791032        

552 	 Chris Fortier
Now Downloading: https://www.allmusic.com/search/artists/Chris%20Fortier
	Chris Fortier                                                         MN0000777416        
	Gary Fortier                                                          MN0001968507        
	Chris Fritter                                                         MN0000627751        
	Chris Stapleton                                                       MN0001766237        
	Chris Connor                                                          MN0000776337        
	Chris Hillman                                                         MN0000096944        
	Chris Tomlin                                                          MN0000100575        
	Chris Brown                                                           MN0000266063        
	Chris Potter                                                          MN0000114459        
	Chris Isaak                                                           MN000077

558 	 Christian Marclay
Now Downloading: https://www.allmusic.com/search/artists/Christian%20Marclay
	Christian Marclay                                                     MN0000123058        
	Charlie Christian                                                     MN0000805930        
	Christian McBride                                                     MN0000103600        
	Christian Scott                                                       MN0000123195        
	Christian Death                                                       MN0000120060        
	Chris Christian                                                       MN0000775206        
	Christian Tetzlaff                                                    MN0002212291        
	Christian Wolff                                                       MN0000680731        
	Christian Thielemann                                                  MN0002322223        
	Christian Lindberg                                                    

564 	 Christine Lavin
Now Downloading: https://www.allmusic.com/search/artists/Christine%20Lavin
	Christine Lavin                                                       MN0000127164        
	Christine Levine                                                      MN0001387403        
	Christine and the Queens                                              MN0003106289        
	Christine McVie                                                       MN0000123915        
	Christine Collister                                                   MN0000781196        
	Christine Schäfer                                                     MN0002178159        
	Christine Schornsheim                                                 MN0002144821        
	Christina Milian                                                      MN0000126168        
	Christine Brewer                                                      MN0001651613        
	Christine Busch                                                       MN00

Ellie Goulding 	 MN0001596642 	 https://www.allmusic.com/artist/ellie-goulding-mn0001596642


# All Music

In [ ]:
#https://www.allmusic.com/search/all/Bliss
#

In [ ]:
savename="bliss.p"

In [ ]:
url = "https://www.allmusic.com/search/artists/Bliss"

In [ ]:
def getAllMusic(url, savename, parse=False, force=False, debug=False, sleeptime=2):
    if isFile(savename):
        if debug:
            print("{0} exists.".format(savename))
        if force is False:
            return False
        else:
            print("Downloading again.")

    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    if debug:
        print("Now Downloading in Artists(): {0}".format(url))
        print("                   Saving as: {0}".format(savename))

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    print("Saving {0} (force={1})".format(savename, force))
    saveJoblib(data=data, filename=savename, compress=True)
    print("Done. Sleeping for {0} seconds".format(sleeptime))
    sleep(sleeptime)

    if isFile(savename):
        return True
    else:
        return False


In [ ]:
getAllMusic(url, savename)

In [ ]:
from webUtils import getHTML

In [ ]:
data = getHTML('bliss.p')

In [ ]:
data

In [ ]:
uls = data.findAll("ul", {"class": "search-results"})

In [ ]:
for ul in uls:
    for li in ul.findAll("li", {"class": "artist"}):
        for div in li.findAll("div", {"class": "name"}):
            link     = div.find("a")
            href     = link.attrs['href']
            tooltip  = link.attrs['data-tooltip']
            artistID = None
            if isinstance(tooltip, dict):
                artistID = tooltip['id']
            
#            print(div,'\n\n')

In [ ]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')

In [ ]:
%load_ext autoreload
%autoreload
artsAM = artistsAM(discAM)

In [ ]:
artsAM.searchAllMusicForArtist('Creed')

In [ ]:
artistRef = "https://www.allmusic.com/artist/creed-warner-mn0003293932"
artistRef = "{0}/discography".format(artistRef)
artistRef
#urllib.parse.urljoin(artistRef) #, "/discography")

In [ ]:
from fsUtils import setFile, isFile, setDir, isDir, mkDir, mkSubDir, setSubFile
from fsUtils import removeFile
from fileUtils import getBasename, getBaseFilename
from ioUtils import getFile, saveFile, saveJoblib
from webUtils import getWebData, getHTML, getURL
from searchUtils import findExt, findPattern
from timeUtils import clock, elapsed, update
from collections import Counter
from math import ceil
from time import sleep
from artist import artist
from discogsUtils import allmusicUtils
import urllib
from urllib.parse import quote

class artistsAM():
    def __init__(self, discog, basedir=None):
        self.disc = discog
        self.name = "artists-allmusic"
        
        self.artist = artist()
        
        ## General Imports
        self.getCodeDir          = self.disc.getCodeDir
        self.getArtistsDir       = self.disc.getArtistsDir
        self.getArtistsDBDir     = self.disc.getArtistsDBDir
        self.getDiscogDBDir      = self.disc.getDiscogDBDir
        self.discogsUtils        = allmusicUtils()
        
        self.prevSearches        = {}
        
        self.modVal = self.disc.getMaxModVal
        
        self.artistIDtoRefData = None
        
        self.starterDir = setDir(self.getCodeDir(), self.name)
        if not isDir(self.starterDir):
            print("Creating {0}".format(self.starterDir))
            mkDir(self.starterDir, debug=True)
        
    
    ###############################################################################
    # Artist Data
    ###############################################################################
    def getData(self, ifile):
        info = self.artist.getData(ifile)
        return info
    
    def getFileData(self, artistID):
        ifile = self.getArtistSavename(artistID, 1)
        info  = self.getData(ifile)
        return info
        
    
    ###############################################################################
    # Artist Info
    ###############################################################################
    def getArtistURL(self, artistRef, page=1):
        
        url = "{0}/discography".format(artistRef)
        return url
        
        if artistRef.startswith("http"):
            return artistRef
        else:
            baseURL = self.disc.discogURL
            url     = urllib.parse.urljoin(baseURL, quote(artistRef))
            return url

            
        if artistRef.startswith("/artist/") is False:
            print("Artist Ref needs to start with /artist/")
            return None
        
        baseURL = self.disc.discogURL
        url     = urllib.parse.urljoin(baseURL, quote(artistRef))
        url     = urllib.parse.urljoin(url, "?sort=year%2Casc&limit=500") ## Make sure we get 500 entries)
        if isinstance(page, int) and page > 1:
            pageURL = "&page={0}".format(page)
            url = "{0}{1}".format(url, pageURL)
        return url
    
    
    def getArtistSavename(self, discID, page=1):
        artistDir = self.disc.getArtistsDir()
        modValue  = self.discogsUtils.getDiscIDHashMod(discID=discID, modval=self.disc.getMaxModVal())
        if modValue is not None:
            outdir    = mkSubDir(artistDir, str(modValue))
            if isinstance(page, int) and page > 1:
                outdir = mkSubDir(outdir, "extra")
                savename  = setFile(outdir, discID+"-{0}.p".format(page))
            else:
                savename  = setFile(outdir, discID+".p")
                
            return savename
        return None
        
    
    ###############################################################################
    # Artist Downloads
    ###############################################################################
    def downloadArtistURL(self, url, savename, parse=False, force=False, debug=False, sleeptime=2):
        if isFile(savename):
            if debug:
                print("{0} exists.".format(savename))
            if force is False:
                return False
            else:
                print("Downloading again.")
                        
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 

        if debug:
            print("Now Downloading in Artists(): {0}".format(url))
            print("                   Saving as: {0}".format(savename))

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        if parse is True:
            info = self.artist.getData(str(data))
            ID = info.ID.ID
            
            if savename.find("/extra/") == -1:
                artistID = getBaseFilename(savename)
            else:
                artistID = getBaseFilename(savename).split("-")[0]
                if ID != artistID:
                    raise ValueError("Problem saving {0} with artistID {1}".format(savename, ID))
                
            if ID != artistID:
                print("  File ID != Artist ID. Renaming to {0}".format(savename))

                savename_test = self.getArtistSavename(ID)
                if force is False and isFile(savename_test):
                    print("{0} exists.".format(savename_test))
                    return False

            
        if force is False and isFile(savename):
            print("{0} exists.".format(savename))
            return True
            
            
        print("Saving {0} (force={1})".format(savename, force))
        saveJoblib(data=data, filename=savename, compress=True)
        print("Done. Sleeping for {0} seconds".format(sleeptime))
        sleep(sleeptime)
        
        if isFile(savename):
            return True
        else:
            return False
        
        
            
    ################################################################################
    # Download Search Artist (2a)
    ################################################################################
    def searchAllMusicForArtist(self, artist, debug=True, sleeptime=2):
        if self.prevSearches.get(artist) is not None:
            return
        if self.prevSearches.get(artist.upper()) is not None:
            return
        self.prevSearches[artist] = True
        
        
        print("\n\n===================== Searching For {0} =====================".format(artist))
        baseURL = self.disc.discogSearchURL
        
        url = urllib.parse.urljoin(baseURL, "{0}{1}".format("artists/", quote(artist)))
        
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 

        sleep(1)

        print("Downloading: {0}".format(url))

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        bsdata = getHTML(data)
        
        artistDB  = {}
        
        uls = bsdata.findAll("ul", {"class": "search-results"})
        for ul in uls:
            lis = ul.findAll("li", {"class": "artist"})
            for li in lis:
                divs = li.findAll("div", {"class": "name"})
                for div in divs:
                    link     = div.find("a")
                    href     = link.attrs['href']
                    tooltip  = link.attrs['data-tooltip']
                    try:
                        from json import loads
                        tooltip = loads(tooltip)
                        artistID = tooltip['id']
                    except:
                        artistID = None
            
                    if artistDB.get(href) is None:
                        artistDB[href] = {"N": 0, "Name": artist}
                    artistDB[href]["N"] += 1
        
    
        if debug:
            print("Found {0} artists".format(len(artistDB)))
                
        iArtist = 0
        for href, hrefData in artistDB.items():
            iArtist += 1
        
            discID   = self.discogsUtils.getArtistID(href)
            url      = self.getArtistURL(href)
            savename = self.getArtistSavename(discID)

            print(iArtist,'/',len(artistDB),'\t:',discID,'\t',url)
            
            if isFile(savename):
                continue

            self.downloadArtistURL(url, savename, sleeptime=sleeptime)
                
            




    ################################################################################
    # Parse Artist Data (3)
    ################################################################################
    def parseArtistFile(ifile):
        bsdata     = getHTML(get(ifile))
        artistData = self.parse(bsdata) 
        return artistData
    
    
    def parseArtistModValExtraFiles(self, modVal, debug=False):
        force=False
        print("Parsing Artist Extra Files For ModVal {0}".format(modVal))
        artistInfo = artist()

        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        dirVal = setDir(dirVal, "extra")
        files  = findExt(dirVal, ext='.p')
        
        if len(files) == 0:
            return
        print("  Found {0} extra files for ModVal {1}".format(len(files), modVal))

        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        
        if force is False:
            print("  Loaded ", end="")
            dbdata = getFile(dbname, version=3)
            print("{0} artist IDs.".format(len(dbdata)))
        else:
            print("Forcing Reloads of ModVal={0}".format(modVal))
            print("  Processing {0} files.".format(len(files)))
            dbdata = {}

        saveIt = 0
        
        nArtistMedia = {}
        
        for j,ifile in enumerate(files):
            if force is True:
                if j % 500 == 0:
                    print("\tProcessed {0}/{1} files.".format(j,len(files)))
                    
            
            info     = artistInfo.getData(ifile)
            artistID = info.ID.ID
            
            #print(artistID,'\t',sum([len(x) for x in dbdata[artistID].media.media.values()]),end="\t")

            keys = list(set(list(info.media.media.keys()) + list(dbdata[artistID].media.media.keys())))
            for k in keys:
                v = info.media.media.get(k)
                if v is None:
                    continue
                iVal  = {v2.code: v2 for v2 in v}
                dVal  = dbdata[artistID].media.media.get(k)
                if dVal is None:
                    Tretval = iVal
                    saveIt += len(iVal)
                else:
                    Tretval = {v2.code: v2 for v2 in dVal}
                    Tretval.update(iVal)
                    saveIt += len(iVal)
                dbdata[artistID].media.media[k] = list(Tretval.values())
                
            print(sum([len(x) for x in dbdata[artistID].media.media.values()]))
            
            
        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} new artist media to {1}".format(saveIt, savename))
            saveJoblib(data=dbdata, filename=savename, compress=True)
            
        return saveIt

    
    
    def parseArtistModValFiles(self, modVal, force=False, debug=False):        
        from os import path
        from datetime import datetime, timedelta

        print("Parsing Artist Files For ModVal {0}".format(modVal))
        artistInfo = artist()

        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        files  = findExt(dirVal, ext='.p')
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))

        
        
        ### Check for recent files
        lastModified = datetime.fromtimestamp(path.getmtime(dbname))
        now    = datetime.now()
        if force is False:
            numRecent = [ifile for ifile in files if datetime.fromtimestamp(path.getmtime(ifile)) > lastModified]
            numNew    = [ifile for ifile in files if (now-datetime.fromtimestamp(path.getmtime(ifile))).days < 1]
            if len(numRecent) == 0:
                print("  ===> Found {0} files, but there are no new files to parse so skipping.".format(len(files)))
                return 0
            else:
                print("  ===> Found {0} files. There are {1} new files to parse.".format(len(files), len(numRecent)))

                
        if force is False:
            dbdata = getFile(dbname, version=3)
        else:
            print("Forcing Reloads of ModVal={0}".format(modVal))
            print("  Processing {0} files.".format(len(files)))
            dbdata = {}

        saveIt = 0
        for j,ifile in enumerate(files):
            if force is True:
                if j % 500 == 0:
                    print("\tProcessed {0}/{1} files.".format(j,len(files)))
            artistID = getBaseFilename(ifile)
            isKnown  = dbdata.get(artistID)
            recent   = datetime.fromtimestamp(path.getctime(ifile))
            if isKnown is None or recent > lastModified:
                saveIt += 1
                info   = artistInfo.getData(ifile)
                
                if info.ID.ID != artistID:
                    print("ID From Name: {0}".format(artistID))
                    print("ID From File: {0}".format(info.ID.ID))

                    print("File: {0}".format(ifile))
                    print("Info: {0}".format(info.url.get()))
                    1/0
                
                dbdata[artistID] = info

        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} new artist IDs to {1}".format(saveIt, savename))
            saveJoblib(data=dbdata, filename=savename, compress=True)
            
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            
        return saveIt
    

    def parseArtistFiles(self, force=False, debug=False):   
        
        totalSaves = 0
        maxModVal  = self.disc.getMaxModVal()
        for modVal in range(maxModVal):
            saveIt = self.parseArtistModValFiles(modVal, force=force, debug=debug)
            totalSaves += saveIt
            
        print("Saved {0} new artist IDs".format(totalSaves))      
        
        
    
    ################################################################################
    # Check ArtistDB Files
    ################################################################################ 
    def rmIDFromDB(self, artistID, modValue=None):
        if modValue is None:
            modValue  = self.discogsUtils.getDiscIDHashMod(discID=artistID, modval=self.disc.getMaxModVal())
        artistDBDir = self.disc.getArtistsDBDir()
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)
        
        saveVal = False

        if isinstance(artistID, str):
            artistID = [artistID]
        elif not isinstance(artistID, list):
            raise ValueError("Not sure what to do with {0}".format(artistID))
            
        for ID in artistID:
            try:
                del dbdata[ID]
                print("Deleted {0}".format(ID))
                saveVal = True
            except:
                print("Not there...")

            try:
                savename = self.getArtistSavename(ID)
                removeFile(savename)
                print("Removed File {0}".format(savename))
            except:
                print("Not there...")

        if saveVal:
            print("Saving {0}".format(dbname))
            saveFile(idata=dbdata, ifile=dbname)
        else:
            print("No reason to save {0}".format(dbname))

    
    def assertDBModValExtraData(self, modVal):
        
        artistDBDir = self.disc.getArtistsDBDir()
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
        dbdata  = getFile(dbname)
        nerrs = 0
        
        for artistID,artistData in dbdata.items():
            pages = artistData.pages
            if pages.more is True:
                npages = pages.pages
                artistRef = artistData.url.url
                for p in range(2, npages+1):
                    url      = self.getArtistURL(artistRef, p)
                    savename = self.getArtistSavename(artistID, p)
                    if not isFile(savename):
                        self.downloadArtistURL(url=url, savename=savename, force=True, debug=True)
                        sleep(2)
                        
            
    def assertDBModValData(self, modVal):
        
        artistDBDir = self.disc.getArtistsDBDir()
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
        dbdata  = getFile(dbname)
        nerrs = 0
        
        if self.artistIDtoRefData is None:
            self.artistIDtoRefData = self.disc.getArtistIDToRefData()
        
        dels = []
        for artistID,artistData in dbdata.items():
            pages = artistData.pages
            if pages.redo is True and False:
                artistRef = artistData.url.url
                url       = self.getArtistURL(artistRef, 1)
                savename  = self.getArtistSavename(artistID, 1)
                self.downloadArtistURL(url=url, savename=savename, force=True, debug=True)

            ID = artistData.ID.ID
            if ID != artistID:

                nerrs += 1

                if "-" in artistID:
                    print("Extra file: {0}".format(artistID))
                    continue
                else:
                    dels.append(artistID)
                    
                    rmsavename = self.getArtistSavename(artistID)


                    ## ID = artistID                    
                    refRef      = self.artistIDtoRefData.get(artistID)
                    if refRef is None:
                        raise ValueError("Ref for ID [{0}] is None!".format(artistID))
                    else:
                        print("ArtistRef:",refRef)
                        urlRef         = self.getArtistURL(refRef)
                        savenameArtRef = self.getArtistSavename(artistID)


                    ## ID = info.ID.ID
                    try:
                        info  = self.getFileData(artistID)
                    except:
                        info  = None

                    if info is not None:
                        try:
                            refIDID      = artistIDtoRefData[info.ID.ID]
                        except:
                            refIDID      = info.url.url
                        print("ArtistID: ",refIDID)
                        urlIDID      = self.getArtistURL(refIDID)
                        savenameIDID = self.getArtistSavename(info.ID.ID)
                    else:
                        refIDID      = None
                        urlIDID      = None
                        savenameIDID = None

                        
                    if isFile(rmsavename):
                        removeFile(rmsavename)


                    if isFile(savenameArtRef):
                        removeFile(savenameArtRef)
                        self.downloadArtistURL(url=urlRef, savename=savenameArtRef, force=True, debug=True)

                    if savenameArtRef != savenameIDID:
                        if isFile(savenameIDID):
                            removeFile(savenameIDID)
                            self.downloadArtistURL(url=urlIDID, savename=savenameIDID, force=True, debug=True)


                    #print(rmsavename,'\t',savenameArtID,'\t',savenameIDID)        
        
        print("Found {0} errors with modVal {1}".format(nerrs, modVal))
        
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        print("Found {0} artist IDs in {1}".format(len(dbdata), dbname))
        
        for artistID in dels:
            print("Deleting {0}".format(artistID))
            try:
                del dbdata[artistID]
            except:
                continue
            
        if len(dels) > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} artist IDs to {1}".format(len(dbdata), savename))
            saveJoblib(data=dbdata, filename=savename, compress=True)
        
        
    
    ################################################################################
    # Collect Metadata About Artists (4)
    ################################################################################
    def createArtistModValMetadata(self, modVal, db=None, debug=False):
        if db is None:
            db = self.disc.getArtistsDBModValData(modVal)
    
        artistIDMetadata = {k: [v.artist.name, v.url.url] for k,v in db.items()}
        
        for artistID,artistData in db.items():
            if artistData.profile.variations is not None:
                artistIDMetadata[artistID].append([v2.name for v2 in artistData.profile.variations])
            else:
                artistIDMetadata[artistID].append([artistData.artist.name])
        
        artistDBDir = self.disc.getArtistsDBDir()     
        savename    = setSubFile(artistDBDir, "metadata", "{0}-Metadata.p".format(modVal))
        
        print("Saving {0} new artist IDs name data to {1}".format(len(artistIDMetadata), savename))
        saveJoblib(data=artistIDMetadata, filename=savename, compress=True)
        
        
    def createArtistAlbumModValMetadata(self, modVal, db=None, debug=False):
        if db is None:
            db = self.disc.getArtistsDBModValData(modVal)
        
        artistIDMetadata = {}
        for artistID,artistData in db.items():
            artistIDMetadata[artistID] = {}
            for mediaName,mediaData in artistData.media.media.items():
                albumURLs  = {mediaValues.code: mediaValues.url for mediaValues in mediaData}
                albumNames = {mediaValues.code: mediaValues.album for mediaValues in mediaData}
                artistIDMetadata[artistID][mediaName] = [albumNames, albumURLs]
        
        artistDBDir = self.disc.getArtistsDBDir()     
        savename    = setSubFile(artistDBDir, "metadata", "{0}-MediaMetadata.p".format(modVal))
        
        print("Saving {0} new artist IDs media data to {1}".format(len(artistIDMetadata), savename))
        saveJoblib(data=artistIDMetadata, filename=savename, compress=True)

In [ ]:
#from discogsUtils import allmusicUtils
# Basic stuff
%load_ext autoreload
%autoreload
from discogsUtils import allmusicUtils

discUtils = allmusicUtils()
artistID = discUtils.getArtistID('https://www.allmusic.com/artist/bliss-mn0001514655', debug=True)
print("retval = ",artistID)

In [ ]:
s = '{"id":"MN0001498391","thumbnail":true}'

In [ ]:
from json import loads
loads(s)

In [ ]:
from ioUtils import getFile
from fsUtils import isFile
from webUtils import getHTML, isBS4
from strUtils import fixName
from math import ceil, floor

from discogsBase import discogs

class artistAMIDClass:
    def __init__(self, ID=None, err=None):
        self.ID=ID
        self.err=err
        
    def get(self):
        return self.__dict__
    
            
class artistAMURLClass:
    def __init__(self, url=None, err=None):
        self.url = url
        self.err = err
        
    def get(self):
        return self.__dict__
        
        
class artistAMNameClass:
    def __init__(self, name=None, err=None):
        self.name = name
        self.err  = err
        
    def get(self):
        return self.__dict__
    

class artistAMMediaClass:
    def __init__(self, err=None):
        self.media = {}
        self.err   = err
        
    def get(self):
        return self.__dict__
    

class artistAMMediaDataClass:
    def __init__(self, album=None, url=None, aclass=None, aformat=None, artist=None, code=None, year=None, err=None):
        self.album   = album
        self.url     = url
        self.aclass  = aclass
        self.aformat = aformat
        self.artist  = artist
        self.code    = code
        self.year    = year
        self.err     = err
        
    def get(self):
        return self.__dict__
    

class artistAMMediaAlbumClass:
    def __init__(self, url=None, album=None, aformat=None, err=None):
        self.url     = url
        self.album   = album
        self.aformat = aformat
        self.err     = err        
        
    def get(self):
        return self.__dict__

    
class artistAMMediaCountsClass:
    def __init__(self, err=None):
        self.counts = {}
        self.err    = err
        
    def get(self):
        return self.__dict__
    

class artistAMPageClass:
    def __init__(self, ppp = None, tot = None, more=None, redo=None, err=None):
        self.ppp   = ppp
        self.tot   = tot
        if isinstance(ppp, int) and isinstance(tot, int):
            self.pages = int(ceil(tot/ppp))
        else:
            self.pages = None

        self.err   = err

        self.more  = more
        self.redo  = redo
        
    def get(self):
        return self.__dict__
    

class artistAMProfileClass:
    def __init__(self, profile=None, aliases=None, members=None, sites=None, groups=None, variations=None, err=None):
        self.profile    = profile
        self.aliases    = aliases
        self.members    = members
        self.sites      = sites
        self.groups     = groups
        self.variations = variations
        self.err        = err
        
    def get(self):
        return self.__dict__
    

class artistAMURLInfo:
    def __init__(self, name=None, url=None, ID=None, err=None):
        self.name = name
        self.url  = url
        self.ID   = ID
        self.err  = err
        
    def get(self):
        return self.__dict__
        

class artistAMDataClass:
    def __init__(self, artist=None, url=None, ID=None, pages=None, profile=None, media=None, mediaCounts=None, err=None):
        self.artist      = artist
        self.url         = url
        self.ID          = ID
        self.pages       = pages
        self.profile     = profile
        self.media       = media
        self.mediaCounts = mediaCounts
        self.err         = err
        
    def get(self):
        return self.__dict__


        
class artistAM(discogs):
    def __init__(self, debug=False):
        self.debug = debug
        
    def getData(self, inputdata):
        if isinstance(inputdata, str):
            if isFile(inputdata):
                try:
                    bsdata = getHTML(getFile(inputdata))
                except:
                    try:
                        bsdata = getHTML(getFile(inputdata, version=2))
                    except:
                        raise ValueError("Cannot read artist file: {0}".format(inputdata))
            else:
                try:
                    bsdata = getHTML(inputdata)
                except:
                    raise ValueError("Not sure about string input: {0} . It is not a file".format(inputdata))
        elif isBS4(inputdata):
            bsdata = inputdata
            pass
        else:
            raise ValueError("Not sure about input type: {0}".format(type(inputdata)))

        self.bsdata = bsdata
        
        return self.parse()
        
        
            
        
    def getNamesAndURLs(self, content):
        data = []
        for ref in content.findAll("a"):
            url    = ref.attrs['href']
            name   = ref.text

            ID = self.getartistAMDiscID(url)
            data.append(artistAMURLInfo(name=name, url=url, ID=ID))
        return data





    #######################################################################################################################################
    ## Artist URL
    #######################################################################################################################################
    def getartistAMURL(self):
        result1 = self.bsdata.find("link", {"rel": "canonical"})
        result2 = self.bsdata.find("link", {"hreflang": "en"})
        if result1 and not result2:
            result = result1
        elif result2 and not result1:
            result = result2
        elif result1 and result2:
            result = result1
        else:        
            auc = artistAMURLClass(err=True)
            return auc

        if result:
            url = result.attrs["href"]
            url = url.replace("https://www.allmusic.com", "")
            if url.find("/artist/") == -1:
                url = None
                auc = artistAMURLClass(url=url, err="NoArtist")
            else:
                auc = artistAMURLClass(url=url)
        else:
            auc = artistAMURLClass(err="NoLink")

        return auc

    

    #######################################################################################################################################
    ## Artist ID
    #######################################################################################################################################                
    def getartistAMDiscID(self, suburl):
        ival = "/artist"
        if isinstance(suburl, artistAMURLClass):
            suburl = suburl.url
        if not isinstance(suburl, str):
            aic = artistAMIDClass(err="NotStr")            
            return aic

        pos = suburl.find(ival)
        if pos == -1:
            aic = artistAMIDClass(err="NotArtist")            
            return aic

        data = suburl[pos+len(ival)+1:]
        pos  = data.rfind("-")
        discIDurl = data[(pos+3):]       
        discID = discIDurl.split("/")[0]
        
        try:
            int(discID)
        except:
            aic = artistAMIDClass(err="NotInt")            
            return aic

        aic = artistAMIDClass(ID=discID)
        return aic
    
    

    #######################################################################################################################################
    ## Artist Name
    #######################################################################################################################################
    def getartistAMName(self):
        artistBios = self.bsdata.findAll("div", {"class": "artist-bio-container"})
        if len(artistBios) > 0:
            for div in artistBios:
                h1 = div.find("h1", {"class": "artist-name"})
                if h1 is not None:
                    artistName = h1.text.strip()
                    if len(artistName) > 0:
                        artist = fixName(artistName)
                        anc = artistAMNameClass(name=artist, err=None)
                    else:
                        anc = artistAMNameClass(name=artist, err="Fix")
                else:
                    anc = artistAMNameClass(err="NoH1")
        else:       
            anc = artistAMNameClass(err=True)
            return anc
        
        return anc
    
    

    #######################################################################################################################################
    ## Artist Media
    #######################################################################################################################################
    def getartistAMMediaAlbum(self, td):
        amac = artistAMMediaAlbumClass()
        for span in td.findAll("span"):
            attrs = span.attrs
            if attrs.get("class"):
                if 'format' in attrs["class"]:
                    albumformat = span.text
                    albumformat = albumformat.replace("(", "")
                    albumformat = albumformat.replace(")", "")
                    amac.format = albumformat
                    continue
            span.replaceWith("")

        ref = td.find("a")
        if ref:
            amac.url   = ref.attrs['href']
            amac.album = ref.text
        else:
            amac.err = "NoText"

        return amac
    
    
    def getartistAMMedia(self):
        amc  = artistAMMediaClass()
        name = "Albums"
        amc.media[name] = []

        tables = self.bsdata.findAll("table")
        for table in tables:
            trs = table.findAll("tr")

            header  = trs[0]
            ths     = header.findAll("th")
            headers = [x.text.strip() for x in ths]

            for tr in trs[1:]:
                tds = tr.findAll("td")

                ## Year
                key  = "Year"
                idx  = headers.index(key)
                year = tds[idx].text.strip()

                ## Title
                key   = "Album"
                idx   = headers.index(key)
                ref   = tds[idx].findAll("a")
                try:
                    refdata = ref[0]
                    url     = refdata.attrs['href']
                    album   = refdata.text.strip()
                    
                    data = url.split("/")[-1]
                    pos  = data.rfind("-")
                    discIDurl = data[(pos+3):]       
                    discID = discIDurl.split("/")[0]

                    try:
                        int(discID)
                        code = discID
                    except:
                        code = None
                        
                except:
                    print("Could not parse {0}".format(ref))

                amdc = artistAMMediaDataClass(album=album, url=url, aclass=None, aformat=None, artist=None, code=code, year=year)
                amc.media[name].append(amdc)

        return amc
    
    

    #######################################################################################################################################
    ## Artist Media Counts
    #######################################################################################################################################        
    def getartistAMMediaCounts(self):
        amcc = artistAMMediaCountsClass()
        amcc.err = "No Counts"
        return amcc
        
        results = self.bsdata.findAll("ul", {"class": "facets_nav"})
        if results is None or len(results) == 0:
            amcc.err = "No Counts"
            return amcc
            
        for result in results:
            for li in result.findAll("li"):
                ref = li.find("a")
                if ref:
                    attrs = ref.attrs
                    span = ref.find("span", {"class": "facet_count"})
                    count = None
                    if span:
                        count = span.text
                        credittype    = attrs.get("data-credit-type")
                        creditsubtype = attrs.get("data-credit-subtype")
                        if credittype and creditsubtype:
                            if amcc.counts.get(credittype) == None:
                                amcc.counts[credittype] = {}
                            if amcc.counts[credittype].get(creditsubtype) == None:
                                try:
                                    amcc.counts[credittype][creditsubtype] = int(count)
                                except:
                                    amcc.counts[credittype][creditsubtype] = count
                                    amcc.err = "Non Int"

        return amcc
    
    

    #######################################################################################################################################
    ## Artist Variations
    #######################################################################################################################################
    def getartistAMProfile(self):       
        result = self.bsdata.find("section", {"class": "basic-info"})   
        if result is None:
            apc = artistAMProfileClass(err="No Profile")
            return apc
           
        data   = {}
       
        members = result.find("div", {"class": "group-members"})
        data["Members"] = [item.text.strip() for item in members.findAll("span")]
       
        genres = result.find("div", {"class": "genre"})
        genre  = self.getNamesAndURLs(genres)
        styles = result.find("div", {"class": "styles"})
        style = self.getNamesAndURLs(genres)
        data["Profile"] = str({'genre': genre, 'style': style})
               
        apc = artistAMProfileClass(profile=data.get("Profile"), aliases=data.get("Aliases"),
                                 members=data.get("Members"), groups=data.get("In Groups"),
                                 sites=data.get("Sites"), variations=data.get("Variations"))
        return apc


    
    #######################################################################################################################################
    ## Artist Pages
    #######################################################################################################################################
    def getartistAMPages(self):
        apc   = artistAMPageClass()
        return apc
        from numpy import ceil
        bsdata = self.bsdata


        pageData = bsdata.find("div", {"class": "pagination bottom"})
        if pageData is None:
            err = "pagination bottom"
            apc = artistAMPageClass(err=err)
            return apc
        else:
            x = pageData.find("strong", {"class": "pagination_total"})
            if x is None:
                err = "pagination_total"
                apc = artistAMPageClass(err=err)
                return apc
            else:
                txt = x.text
                txt = txt.strip()
                txt = txt.replace("\n", "")
                retval = [tmp.strip() for tmp in txt.split('of')]

                try:
                    ppp   = int(retval[0].split('–')[-1])
                    tot   = int(retval[1].replace(",", ""))
                except:
                    err   = "int"
                    apc   = artistAMPageClass(err=err)
                    return apc

                if ppp < 500:
                    if tot < 25 or ppp == tot:
                        apc   = artistAMPageClass(ppp=ppp, tot=tot, redo=False, more=False)
                    else:
                        apc   = artistAMPageClass(ppp=ppp, tot=tot, redo=True, more=False)
                else:
                    if tot < 500:
                        apc   = artistAMPageClass(ppp=ppp, tot=tot, redo=False, more=False)
                    else:
                        apc   = artistAMPageClass(ppp=ppp, tot=tot, redo=False, more=True)
                        
                return apc
            
        return artistAMPageClass()



    def parse(self):
        bsdata = self.bsdata
        
        artist      = self.getartistAMName()
        url         = self.getartistAMURL()
        ID          = self.getartistAMDiscID(url)
        pages       = self.getartistAMPages()
        profile     = self.getartistAMProfile()
        mediaCounts = self.getartistAMMediaCounts()
        media       = self.getartistAMMedia()
        
        err = [artist.err, url.err, ID.err, pages.err, profile.err, mediaCounts.err, media.err]
        
        adc = artistAMDataClass(artist=artist, url=url, ID=ID, pages=pages, profile=profile, mediaCounts=mediaCounts, media=media, err=err)
        
        return adc

In [ ]:
file = "/Volumes/Music/Discog/artists-allmusic/68/0000782768.p"
artAM = artistAM()
data = artAM.getData(file)

In [ ]:
data.artist.get()

In [ ]:
data.profile.get()

In [ ]:
data.media.media["Albums"][0].get()

In [ ]:
from searchUtils import findWalkDirExt

In [ ]:
dirval = "/Volumes/Music/Discog/albums/0"

In [ ]:
files = findWalkExt(dirval, ".p")

In [ ]:
len(files)

In [ ]:
files